<a href="https://colab.research.google.com/github/elchupacabris/DNAShot/blob/main/Firebase_Hole_Data_Upload_Fix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Default title text
import pandas as pd
from datetime import datetime
import numpy as np

# --- Firebase Admin SDK Initialization ---
# This part is crucial for interacting with Firestore.
# Ensure you have installed the Firebase Admin SDK: pip install firebase-admin
import firebase_admin
from firebase_admin import credentials, firestore

# # Initialize Firebase Admin SDK if it hasn't been initialized yet
# # Replace 'path/to/your/serviceAccountKey.json' with the actual path to your Firebase service account key
# try:
#     # Check if a default app is already initialized to avoid re-initialization errors
#     firebase_admin.get_app()
# except ValueError:
#     # If not initialized, create credentials and initialize the app
#     # NOTE: Make sure 'serviceAccountKey.json' is in your Colab environment or specify the correct path
#     cred = credentials.Certificate('/content/drive/MyDrive/serviceAccountKey.json')
#     firebase_admin.initialize_app(cred)

# # Get the Firestore client
# db = firestore.client()

# --- DataFrame Loading ---
# Load your data into deposit_df and index_df
# Use the provided path from Google Drive
file_path = "/content/drive/MyDrive/MASTER-FIREBASE.xlsx"

# IMPORTANT: If your data is on different sheets within the same Excel file,
# specify the sheet_name parameter (e.g., sheet_name='HoleDataSheet').
# If they are separate files, update the file paths accordingly.
try:
    # Assuming 'deposit_df' contains the hole-by-hole data
    deposit_df = pd.read_excel(file_path, sheet_name='ALLDATA') # Adjust sheet_name as needed
    print("✅ deposit_df loaded successfully.")
except FileNotFoundError:
    print(f"❌ Error: '{file_path}' not found for deposit_df. Please check the file path.")
    exit()
except Exception as e:
    print(f"❌ Error loading deposit_df: {e}")
    exit()

try:
    # Assuming 'index_df' contains the scorecard summary data
    index_df = pd.read_excel(file_path, sheet_name='INDEX') # Adjust sheet_name as needed
    print("✅ index_df loaded successfully.")
except FileNotFoundError:
    print(f"❌ Error: '{file_path}' not found for index_df. Please check the file path.")
    exit()
except Exception as e:
    print(f"❌ Error loading index_df: {e}")
    exit()

# # --- User ID Mapping ---
# # Map player names to user IDs
# user_id_map = {
#     'ML': '998b30f58115491fbf4188450158',
#     'SM': 'a220a71737234036a6bfb0618f3e'
# }


# # print(f"⬆️ Uploading data to Firestore...")

# # Clean column names in deposit_df and index_df by stripping leading/trailing spaces
# deposit_df.columns = deposit_df.columns.str.strip()
# index_df.columns = index_df.columns.str.strip()

# # Ensure 'SCORECARD DATE' is datetime objects in both dataframes for consistent merging and processing
# # Use errors='coerce' to turn unparseable dates into NaT (Not a Time)
# deposit_df['SCORECARD DATE'] = pd.to_datetime(deposit_df['SCORECARD DATE'], errors='coerce').dt.date
# index_df['SCORECARD DATE'] = pd.to_datetime(index_df['SCORECARD DATE'], errors='coerce').dt.date


# # Drop rows with invalid dates if necessary (or handle them in the loop)
# deposit_df.dropna(subset=['SCORECARD DATE'], inplace=True)
# index_df.dropna(subset=['SCORECARD DATE'], inplace=True)


# # Add a 'userId' column to both dataframes based on the mapping
# deposit_df['userId'] = deposit_df['PLAYER'].map(user_id_map)
# index_df['userId'] = index_df['PLAYER'].map(user_id_map)

# # Drop rows where userId could not be mapped
# deposit_df.dropna(subset=['userId'], inplace=True)
# index_df.dropna(subset=['userId'], inplace=True)


# # Merge index_df and deposit_df to get the SCORE from index_df for correct round ID matching and data inclusion
# merged_hole_data = pd.merge(
#     deposit_df,
#     index_df[['PLAYER', 'CC', 'SCORECARD DATE', 'ROUNDS', 'SCORE', 'userId']],
#     on=['PLAYER', 'CC', 'SCORECARD DATE', 'ROUNDS', 'userId'],
#     how='left',
#     suffixes=('_hole', '_scorecard') # Use suffixes to differentiate score/rounds columns
# )

# # Drop rows where the merge failed to find a matching round in index_df (SCORE_scorecard will be NaN)
# merged_hole_data.dropna(subset=['SCORE_scorecard'], inplace=True)


# # Group by userId and the components that form the round document ID
# # Grouping by userId, CC, Date, Rounds (from deposit_df), and Score (from index_df)
# round_groups = merged_hole_data.groupby(['userId', 'CC', 'SCORECARD DATE', 'ROUNDS', 'SCORE_scorecard'])


# # print(f"Iterating through {len(round_groups)} unique rounds to upload data...")

# # for (userId, cc, date, rounds_hole, score_scorecard), hole_data_group in round_groups:
# #     # Generate the round document ID based on the structure: {Course}_{Date}_{RoundNumber}_{ScoreFromIndexDF}
# #     date_str = date.strftime('%Y-%m-%d') if pd.notna(date) else "N_A_date"
# #     rounds_str = str(int(rounds_hole)) if pd.notna(rounds_hole) and pd.api.types.is_integer(rounds_hole) else str(rounds_hole)
# #     score_str = str(int(score_scorecard)) if pd.notna(score_scorecard) and pd.api.types.is_integer(score_scorecard) else str(score_scorecard)

# #     round_id = f"{cc}_{date_str}_{rounds_str}_{score_str}"

# #     # Construct the path for the round document under the user's rounds collection
# #     round_doc_path = f"players/{userId}/rounds/{round_id}"


# #     # Get a reference to the round document
# #     round_ref = db.document(round_doc_path)

# #     # Aggregate summary data for the round from index_df
# #     # Find the corresponding row in index_df using the grouping keys
# #     round_summary_data_row = index_df[
# #         (index_df['userId'] == userId) &
# #         (index_df['CC'] == cc) &
# #         (index_df['SCORECARD DATE'] == date) &
# #         (index_df['ROUNDS'] == rounds_hole) &
# #         (index_df['SCORE'] == score_scorecard)
# #     ]

# #     round_summary_data = {}
# #     if not round_summary_data_row.empty:
# #         # Take the first row if multiple match (should ideally be unique)
# #         summary_row = round_summary_data_row.iloc[0]
# #         # Prepare summary data dictionary (excluding columns that will be in hole data or are redundant)
# #         round_summary_data = summary_row.drop(columns=[
# #             'PLAYER', 'CC', 'SCORECARD DATE', 'ROUNDS', 'SCORE', 'userId'
# #         ], errors='ignore').to_dict()
# #         round_summary_data['total_score'] = int(summary_row['SCORE']) # Add total score explicitly
# #         round_summary_data['date'] = date_str # Add date as string
# #         round_summary_data['course'] = cc # Add course explicitly
# #         round_summary_data['round_number'] = rounds_str # Add round number explicitly


# #     # Set the round document in Firestore with error handling
# #     try:
# #         round_ref.set(round_summary_data, merge=True) # Use merge=True to avoid overwriting existing data
# #         print(f"\n  Successfully set round document: {round_doc_path}")

# #         # Get a reference to the 'holes' subcollection under this round document
# #         holes_subcollection_ref = round_ref.collection("holes")

# #         # Iterate through each row (each hole) within the current round's group DataFrame
# #         print(f"\n  Uploading {len(hole_data_group)} holes for round: {round_id}")
# #         for _, hole_row in hole_data_group.iterrows():
# #             hole_num = hole_row.get("HOLE") # Hole number from deposit_df (merged)
# #             if pd.isna(hole_num):
# #                 print(f"    Skipping hole data for a row in round {round_id} due to missing HOLE number.")
# #                 continue # Skip if hole number is missing

# #             hole_num_str = str(int(hole_num)) if pd.api.types.is_integer(hole_num) else str(hole_num)

# #             # Prepare data for the hole document
# #             # Include relevant columns from the merged row (hole data)
# #             hole_data = hole_row.drop(columns=[
# #                 'ROUNDS', 'SCORECARD DATE', 'CC', 'PLAYER', 'userId', # Exclude grouping/parent key columns from hole data
# #                 'SCORE_hole', 'SCORE_scorecard' # Exclude other potentially redundant columns
# #             ], errors='ignore').to_dict()

# #             # Convert potential numpy types to native Python types and handle NaN/None
# #             cleaned_hole_data = {}
# #             for key, value in hole_data.items():
# #                 if pd.isna(value):
# #                     cleaned_hole_data[key] = None # Store NaN as None
# #                 # Convert datetime.date/Timestamp objects to string explicitly
# #                 elif isinstance(value, (datetime, pd.Timestamp)):
# #                     cleaned_hole_data[key] = value.strftime('%Y-%m-%d')
# #                 elif isinstance(value, (np.generic, np.ndarray)): # Handle NumPy types
# #                     # Convert NumPy types to native Python types (scalar or list)
# #                     cleaned_hole_data[key] = value.item() if value.size == 1 else value.tolist()
# #                 elif isinstance(value, pd.Series): # Handle potential Series objects
# #                      # Assuming the Series only contains one value or we only need the first
# #                     cleaned_hole_data[key] = value.iloc[0] if not value.empty and not pd.isna(value.iloc[0]) else None
# #                 else:
# #                     cleaned_hole_data[key] = value # Keep other types as is

# #             # Reference to the hole document in the 'holes' subcollection
# #             hole_doc_ref = holes_subcollection_ref.document(hole_num_str)

# #             # Set the data in Firestore
# #             try:
# #                 hole_doc_ref.set(cleaned_hole_data)
# #             except Exception as e:
# #                 print(f"    ❌ Error uploading hole {hole_num_str} for round {round_id}: {e}")

# #     except Exception as e:
# #         print(f"  ❌ Error accessing/processing round document {round_id}: {e}")


# # print(f"\n✅ Upload of data to 'players/{{userId}}/rounds/{{roundId}}' and 'players/{{userId}}/rounds/{{roundId}}/holes/{{holeNumber}}' complete!")

❌ Error: '/content/drive/MyDrive/MASTER-FIREBASE.xlsx' not found for deposit_df. Please check the file path.
❌ Error: '/content/drive/MyDrive/MASTER-FIREBASE.xlsx' not found for index_df. Please check the file path.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(merged_hole_data.columns)

Index(['ROUNDS', 'SCORECARD DATE', 'CC', 'TEE', 'HOLE', 'YDS', 'HDCP', 'PAR',
       'SCORE_hole', 'PUTT', 'DD', 'DDTYPE', 'PINDIST', '1STPU', 'PLAYER',
       'ESC', 'SCORE_scorecard'],
      dtype='object')


# Task
Upload the data from "/content/drive/MyDrive/MASTER-FIREBASE.xlsx" to Firestore, structuring it under the path "/users/{userId}/course_summaries/{courseId}" for course summaries and "/users/{userId}/course_summaries/{courseId}/holes/{holeNumber}" for hole-by-hole data, using the service account key from "/content/drive/MyDrive/serviceAccountKey.json".

## Identify user and course

### Subtask:
Determine how to extract the `userId` and `courseId` from your data or define them based on your application's logic.


**Reasoning**:
Examine the columns of `deposit_df` and `index_df` to identify potential `userId` and `courseId` based on the available data. The previous step already loaded the data and cleaned some columns. The `merged_hole_data` dataframe contains information from both original dataframes.



In [ ]:
print("Columns in deposit_df:")
print(deposit_df.columns)
print("\nColumns in index_df:")
print(index_df.columns)
print("\nSample data from deposit_df (first 5 rows):")
display(deposit_df.head())
print("\nSample data from index_df (first 5 rows):")
display(index_df.head())

Columns in deposit_df:
Index(['ROUNDS', 'SCORECARD DATE', 'CC', 'TEE', 'HOLE', 'YDS', 'HDCP', 'PAR',
       'SCORE', 'PUTT', 'DD', 'DDTYPE', 'PINDIST', '1STPU', 'PLAYER', 'ESC'],
      dtype='object')

Columns in index_df:
Index(['ROUNDS', 'SCORECARD DATE', 'MES', 'YEAR', 'CC', 'PLAYER', 'SCORE',
       'ESC', 'PUTT', 'DD', 'FIR', 'PINDIST', '1STPU', 'GIR', 'SCRAM', '1PUTT',
       '2PUTT', '3PUTT', '4PUTT', '1STPUTT', '1STPUTT3', '1STPUTT4',
       '1STPUTT5', 'GIR2', 'GIR3', 'GIR4', 'GIR5', 'BIRDIE3', 'BIRDIE4',
       'BIRDIE5', 'PAR3', 'PAR4', 'PAR5', 'BOGEY3', 'BOGEY4', 'BOGEY5',
       'DOBLE3', 'DOBLE4', 'DOBLE5', 'TRIPLEOR3', 'TRIPLEOR4', 'TRIPLEOR5',
       'DDMAX', 'DDMAXFIR', 'IDA', 'VUELTA', 'MEJOR', 'DIF+/-'],
      dtype='object')

Sample data from deposit_df (first 5 rows):


,ROUNDS,SCORECARD DATE,CC,TEE,HOLE,YDS,HDCP,PAR,SCORE,PUTT,DD,DDTYPE,PINDIST,1STPU,PLAYER,ESC
0,1,2023-04-07,GUAY#2,BLANCA,1,554,6,5,4,1,NaN,NaN,185.0,NaN,ML,4
1,1,2023-04-07,GUAY#2,BLANCA,2,427,2,4,4,1,NaN,NaN,142.0,NaN,ML,4
2,1,2023-04-07,GUAY#2,BLANCA,3,370,14,4,5,2,NaN,NaN,123.0,NaN,ML,5
3,1,2023-04-07,GUAY#2,BLANCA,4,167,16,3,4,1,NaN,NaN,NaN,NaN,ML,4
4,1,2023-04-07,GUAY#2,BLANCA,5,397,4,4,8,3,NaN,NaN,79.0,NaN,ML,6



Sample data from index_df (first 5 rows):


,ROUNDS,SCORECARD DATE,MES,YEAR,CC,PLAYER,SCORE,ESC,PUTT,DD,...,DOBLE5,TRIPLEOR3,TRIPLEOR4,TRIPLEOR5,DDMAX,DDMAXFIR,IDA,VUELTA,MEJOR,DIF+/-
0,1,2023-04-07,ABRIL,2023,GUAY#2,ML,86,84,31,NaN,...,0,0,1,0,0,-,44,42,Vuelta,-2
1,2,2023-04-07,ABRIL,2023,GUAY#2,SM,87,86,33,NaN,...,0,0,0,1,0,-,47,40,Vuelta,-7
2,3,2023-05-03,MAYO,2023,FUNDADORES,ML,100,93,35,NaN,...,1,1,3,1,0,-,43,57,Ida,14
3,4,2023-05-03,MAYO,2023,FUNDADORES,SM,95,92,32,NaN,...,1,1,2,0,0,-,44,51,Ida,7
4,5,2023-05-17,MAYO,2023,FUNDADORES,ML,97,94,36,NaN,...,2,1,1,0,0,-,47,50,Ida,3


## Modify data grouping

### Subtask:
Adjust the code to group the `merged_hole_data` by user and course.


## Construct firestore paths

### Subtask:
Update the code to construct the Firestore document paths based on the new structure: `/users/{userId}/course_summaries/{courseId}` for the course summary document and `/users/{userId}/course_summaries/{courseId}/holes/{holeNumber}` for the hole documents.


**Reasoning**:
Modify the existing code to construct the new Firestore document paths based on the user, course, and hole number, and update the Firestore set operations to use these new paths.



In [ ]:
if 'index_df' not in locals() or index_df.empty:
    print("❌ index_df not found or is empty. Cannot proceed with hole data upload.")
else:
    # Clean column names in index_df by stripping leading/trailing spaces
    index_df.columns = index_df.columns.str.strip()

    # Ensure 'SCORECARD DATE' is datetime objects in both dataframes for consistent merging
    # Use errors='coerce' to turn unparseable dates into NaT (Not a Time)
    index_df['SCORECARD DATE'] = pd.to_datetime(index_df['SCORECARD DATE'], errors='coerce').dt.date
    deposit_df['SCORECARD DATE'] = pd.to_datetime(deposit_df['SCORECARD DATE'], errors='coerce').dt.date

    # Drop rows with invalid dates if necessary (or handle them in the loop)
    index_df.dropna(subset=['SCORECARD DATE'], inplace=True)
    deposit_df.dropna(subset=['SCORECARD DATE'], inplace=True)

    # Merge index_df and deposit_df to get the SCORE from index_df for correct round ID matching
    merged_hole_data = pd.merge(
        deposit_df,
        index_df[['PLAYER', 'CC', 'SCORECARD DATE', 'ROUNDS', 'SCORE']],
        on=['PLAYER', 'CC', 'SCORECARD DATE', 'ROUNDS'],
        how='left',
        suffixes=('_hole', '_scorecard') # Use suffixes to differentiate score/rounds columns
    )

    # Group by the components that form the round document key components (User and Course)
    round_groups = merged_hole_data.groupby(['PLAYER', 'CC'])

    print(f"Iterating through {len(round_groups)} unique user-course combinations to upload data...")

    for (player, cc), user_course_group in round_groups:
        # Construct the path for the course summary document
        course_summary_path = f"users/{player}/course_summaries/{cc}"

        # Construct the base path for the holes subcollection
        holes_base_path = f"{course_summary_path}/holes"

        # Get a reference to the course summary document
        course_summary_ref = db.document(course_summary_path)

        # Prepare data for the course summary document (can be aggregated from index_df if needed)
        # For this subtask, we will create a minimal document to ensure the path exists
        # In a real scenario, you might want to put aggregated stats here.
        course_summary_data = {
            "player": player,
            "course": cc,
            "last_updated": firestore.SERVER_TIMESTAMP # Example field
        }

        # Set the course summary document in Firestore
        try:
            course_summary_ref.set(course_summary_data, merge=True) # Use merge=True to avoid overwriting existing data
            print(f"\n  Successfully set course summary document: {course_summary_path}")

            # Get a reference to the 'holes' subcollection under this course summary document
            holes_subcollection_ref = course_summary_ref.collection("holes")

            # Iterate through each row (each hole) within the current user-course group DataFrame
            print(f"\n  Uploading {len(user_course_group)} holes for user: {player}, course: {cc}")
            for _, hole_row in user_course_group.iterrows():
                hole_num = hole_row.get("HOLE") # Hole number from deposit_df (merged)
                if pd.isna(hole_num):
                    print(f"    Skipping hole data for a row in user {player}, course {cc} due to missing HOLE number.")
                    continue # Skip if hole number is missing

                hole_num_str = str(int(hole_num)) if pd.api.types.is_integer(hole_num) else str(hole_num)

                # Prepare data for the hole document
                # Include relevant columns from the merged row (hole data)
                hole_data = hole_row.drop(columns=[
                    'PLAYER', 'CC', # Exclude grouping/parent key columns from hole data
                    'SCORECARD DATE', 'ROUNDS', 'SCORE_hole', 'SCORE_scorecard' # Exclude other potentially redundant columns
                ], errors='ignore').to_dict()

                # Convert potential numpy types to native Python types and handle NaN/None
                cleaned_hole_data = {}
                for key, value in hole_data.items():
                    if pd.isna(value):
                        cleaned_hole_data[key] = None # Store NaN as None
                    # Convert datetime.date/Timestamp objects to string explicitly
                    elif isinstance(value, (datetime, pd.Timestamp)):
                        cleaned_hole_data[key] = value.strftime('%Y-%m-%d')
                    elif isinstance(value, (np.generic, np.ndarray)): # Handle NumPy types
                        # Convert NumPy types to native Python types (scalar or list)
                        cleaned_hole_data[key] = value.item() if value.size == 1 else value.tolist()
                    elif isinstance(value, pd.Series): # Handle potential Series objects
                        cleaned_hole_data[key] = value.iloc[0] if not pd.isna(value.iloc[0]) else None
                    else:
                        cleaned_hole_data[key] = value # Keep other types as is

                # Reference to the hole document in the 'holes' subcollection
                hole_doc_ref = holes_subcollection_ref.document(hole_num_str)

                # Set the data in Firestore
                try:
                    hole_doc_ref.set(cleaned_hole_data)
                except Exception as e:
                    print(f"    ❌ Error uploading hole {hole_num_str} for user {player}, course {cc}: {e}")

        except Exception as e:
            print(f"  ❌ Error accessing/processing user {player}, course {cc}: {e}")

print(f"\n✅ Upload of data to 'users/{{userId}}/course_summaries/{{courseId}}' and 'users/{{userId}}/course_summaries/{{courseId}}/holes/{{holeNumber}}' complete!")

Iterating through 44 unique user-course combinations to upload data...
  ❌ Error accessing/processing user ML, course ARRAYANES: 403 Permission denied on resource project golf-scorecard-1847c. [reason: "CONSUMER_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "firestore.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "golf-scorecard-1847c"
}
metadata {
  key: "consumer"
  value: "projects/golf-scorecard-1847c"
}
, locale: "en-US"
message: "Permission denied on resource project golf-scorecard-1847c."
, links {
  description: "Google developers console"
  url: "https://console.developers.google.com"
}
]
  ❌ Error accessing/processing user ML, course BRICEÑO: 403 Permission denied on resource project golf-scorecard-1847c. [reason: "CONSUMER_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "firestore.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "golf-scorecard-1847c"
}
metadata {
  key: "consumer"
  value: "projects/

## Upload course summary data

### Subtask:
Upload the relevant summary data for each user and course to the `/users/{userId}/course_summaries/{courseId}` document.


**Reasoning**:
Populate the course_summary_data dictionary with relevant summary information from the index_df, and include error handling for the Firestore set operation.



In [ ]:
from firebase_admin import firestore

if 'index_df' not in locals() or index_df.empty:
    print("❌ index_df not found or is empty. Cannot proceed with data upload.")
else:
    # Clean column names in index_df by stripping leading/trailing spaces
    index_df.columns = index_df.columns.str.strip()

    # Ensure 'SCORECARD DATE' is datetime objects in both dataframes for consistent merging
    # Use errors='coerce' to turn unparseable dates into NaT (Not a Time)
    index_df['SCORECARD DATE'] = pd.to_datetime(index_df['SCORECARD DATE'], errors='coerce').dt.date
    deposit_df['SCORECARD DATE'] = pd.to_datetime(deposit_df['SCORECARD DATE'], errors='coerce').dt.date

    # Drop rows with invalid dates if necessary (or handle them in the loop)
    index_df.dropna(subset=['SCORECARD DATE'], inplace=True)
    deposit_df.dropna(subset=['SCORECARD DATE'], inplace=True)

    # Merge index_df and deposit_df to get the SCORE from index_df for correct round ID matching
    merged_hole_data = pd.merge(
        deposit_df,
        index_df[['PLAYER', 'CC', 'SCORECARD DATE', 'ROUNDS', 'SCORE']],
        on=['PLAYER', 'CC', 'SCORECARD DATE', 'ROUNDS'],
        how='left',
        suffixes=('_hole', '_scorecard') # Use suffixes to differentiate score/rounds columns
    )

    # Group by the components that form the round document key components (User and Course)
    user_course_groups = merged_hole_data.groupby(['PLAYER', 'CC'])

    print(f"Iterating through {len(user_course_groups)} unique user-course combinations to upload data...")

    for (player, cc), user_course_group in user_course_groups:
        # Construct the path for the course summary document
        course_summary_path = f"users/{player}/course_summaries/{cc}"

        # Get a reference to the course summary document
        course_summary_ref = db.document(course_summary_path)

        # Aggregate summary data from index_df for the current player and course
        player_course_index_df = index_df[(index_df['PLAYER'] == player) & (index_df['CC'] == cc)]

        # Prepare data for the course summary document
        course_summary_data = {
            "player": player,
            "course": cc,
            "number_of_rounds": len(player_course_index_df),
            "average_score": player_course_index_df['SCORE'].mean() if not player_course_index_df.empty else None,
            "last_updated": firestore.SERVER_TIMESTAMP # Example field
        }

        # Set the course summary document in Firestore with error handling
        try:
            course_summary_ref.set(course_summary_data, merge=True) # Use merge=True to avoid overwriting existing data
            print(f"\n  Successfully set course summary document: {course_summary_path}")

            # Get a reference to the 'holes' subcollection under this course summary document
            holes_subcollection_ref = course_summary_ref.collection("holes")

            # Iterate through each row (each hole) within the current user-course group DataFrame
            print(f"\n  Uploading {len(user_course_group)} holes for user: {player}, course: {cc}")
            for _, hole_row in user_course_group.iterrows():
                hole_num = hole_row.get("HOLE") # Hole number from deposit_df (merged)
                if pd.isna(hole_num):
                    print(f"    Skipping hole data for a row in user {player}, course {cc} due to missing HOLE number.")
                    continue # Skip if hole number is missing

                hole_num_str = str(int(hole_num)) if pd.api.types.is_integer(hole_num) else str(hole_num)

                # Prepare data for the hole document
                # Include relevant columns from the merged row (hole data)
                hole_data = hole_row.drop(columns=[
                    'PLAYER', 'CC', # Exclude grouping/parent key columns from hole data
                    'SCORECARD DATE', 'ROUNDS', 'SCORE_hole', 'SCORE_scorecard' # Exclude other potentially redundant columns
                ], errors='ignore').to_dict()

                # Convert potential numpy types to native Python types and handle NaN/None
                cleaned_hole_data = {}
                for key, value in hole_data.items():
                    if pd.isna(value):
                        cleaned_hole_data[key] = None # Store NaN as None
                    # Convert datetime.date/Timestamp objects to string explicitly
                    elif isinstance(value, (datetime, pd.Timestamp)):
                        cleaned_hole_data[key] = value.strftime('%Y-%m-%d')
                    elif isinstance(value, (np.generic, np.ndarray)): # Handle NumPy types
                        # Convert NumPy types to native Python types (scalar or list)
                        cleaned_hole_data[key] = value.item() if value.size == 1 else value.tolist()
                    elif isinstance(value, pd.Series): # Handle potential Series objects
                        cleaned_hole_data[key] = value.iloc[0] if not pd.isna(value.iloc[0]) else None
                    else:
                        cleaned_hole_data[key] = value # Keep other types as is

                # Reference to the hole document in the 'holes' subcollection
                hole_doc_ref = holes_subcollection_ref.document(hole_num_str)

                # Set the data in Firestore
                try:
                    hole_doc_ref.set(cleaned_hole_data)
                except Exception as e:
                    print(f"    ❌ Error uploading hole {hole_num_str} for user {player}, course {cc}: {e}")

        except Exception as e:
            print(f"  ❌ Error accessing/processing user {player}, course {cc}: {e}")

print(f"\n✅ Upload of data to 'users/{{userId}}/course_summaries/{{courseId}}' and 'users/{{userId}}/course_summaries/{{courseId}}/holes/{{holeNumber}}' complete!")


Iterating through 44 unique user-course combinations to upload data...
  ❌ Error accessing/processing user ML, course ARRAYANES: 403 Permission denied on resource project golf-scorecard-1847c. [reason: "CONSUMER_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "firestore.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "golf-scorecard-1847c"
}
metadata {
  key: "consumer"
  value: "projects/golf-scorecard-1847c"
}
, locale: "en-US"
message: "Permission denied on resource project golf-scorecard-1847c."
, links {
  description: "Google developers console"
  url: "https://console.developers.google.com"
}
]
  ❌ Error accessing/processing user ML, course BRICEÑO: 403 Permission denied on resource project golf-scorecard-1847c. [reason: "CONSUMER_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "firestore.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "golf-scorecard-1847c"
}
metadata {
  key: "consumer"
  value: "projects/

## Upload hole data to subcollection

### Subtask:
Iterate through the grouped hole data and upload each hole's information to the 'holes' subcollection under the corresponding course summary document.


**Reasoning**:
Iterate through the grouped hole data and upload each hole's information to the 'holes' subcollection under the corresponding course summary document. This involves extracting hole data, preparing it for Firestore, and setting the document within the subcollection with error handling.



In [ ]:
from firebase_admin import firestore
from datetime import datetime
import numpy as np
import pandas as pd

if 'index_df' not in locals() or index_df.empty:
    print("❌ index_df not found or is empty. Cannot proceed with data upload.")
else:
    # Clean column names in index_df by stripping leading/trailing spaces
    index_df.columns = index_df.columns.str.strip()

    # Ensure 'SCORECARD DATE' is datetime objects in both dataframes for consistent merging
    # Use errors='coerce' to turn unparseable dates into NaT (Not a Time)
    index_df['SCORECARD DATE'] = pd.to_datetime(index_df['SCORECARD DATE'], errors='coerce').dt.date
    deposit_df['SCORECARD DATE'] = pd.to_datetime(deposit_df['SCORECARD DATE'], errors='coerce').dt.date

    # Drop rows with invalid dates if necessary (or handle them in the loop)
    index_df.dropna(subset=['SCORECARD DATE'], inplace=True)
    deposit_df.dropna(subset=['SCORECARD DATE'], inplace=True)

    # Merge index_df and deposit_df to get the SCORE from index_df for correct round ID matching
    merged_hole_data = pd.merge(
        deposit_df,
        index_df[['PLAYER', 'CC', 'SCORECARD DATE', 'ROUNDS', 'SCORE']],
        on=['PLAYER', 'CC', 'SCORECARD DATE', 'ROUNDS'],
        how='left',
        suffixes=('_hole', '_scorecard') # Use suffixes to differentiate score/rounds columns
    )

    # Group by the components that form the round document key components (User and Course)
    user_course_groups = merged_hole_data.groupby(['PLAYER', 'CC'])

    print(f"Iterating through {len(user_course_groups)} unique user-course combinations to upload data...")

    for (player, cc), user_course_group in user_course_groups:
        # Construct the path for the course summary document
        course_summary_path = f"users/{player}/course_summaries/{cc}"

        # Get a reference to the course summary document
        course_summary_ref = db.document(course_summary_path)

        # Aggregate summary data from index_df for the current player and course
        player_course_index_df = index_df[(index_df['PLAYER'] == player) & (index_df['CC'] == cc)]

        # Prepare data for the course summary document
        course_summary_data = {
            "player": player,
            "course": cc,
            "number_of_rounds": len(player_course_index_df),
            "average_score": player_course_index_df['SCORE'].mean() if not player_course_index_df.empty else None,
            "last_updated": firestore.SERVER_TIMESTAMP # Example field
        }

        # Set the course summary document in Firestore with error handling
        try:
            course_summary_ref.set(course_summary_data, merge=True) # Use merge=True to avoid overwriting existing data
            print(f"\n  Successfully set course summary document: {course_summary_path}")

            # Get a reference to the 'holes' subcollection under this course summary document
            holes_subcollection_ref = course_summary_ref.collection("holes")

            # Iterate through each row (each hole) within the current user-course group DataFrame
            print(f"\n  Uploading {len(user_course_group)} holes for user: {player}, course: {cc}")
            for _, hole_row in user_course_group.iterrows():
                hole_num = hole_row.get("HOLE") # Hole number from deposit_df (merged)
                if pd.isna(hole_num):
                    print(f"    Skipping hole data for a row in user {player}, course {cc} due to missing HOLE number.")
                    continue # Skip if hole number is missing

                hole_num_str = str(int(hole_num)) if pd.api.types.is_integer(hole_num) else str(hole_num)

                # Prepare data for the hole document
                # Include relevant columns from the merged row (hole data)
                hole_data = hole_row.drop(columns=[
                    'PLAYER', 'CC', # Exclude grouping/parent key columns from hole data
                    'SCORECARD DATE', 'ROUNDS', 'SCORE_hole', 'SCORE_scorecard' # Exclude other potentially redundant columns
                ], errors='ignore').to_dict()

                # Convert potential numpy types to native Python types and handle NaN/None
                cleaned_hole_data = {}
                for key, value in hole_data.items():
                    if pd.isna(value):
                        cleaned_hole_data[key] = None # Store NaN as None
                    # Convert datetime.date/Timestamp objects to string explicitly
                    elif isinstance(value, (datetime, pd.Timestamp)):
                        cleaned_hole_data[key] = value.strftime('%Y-%m-%d')
                    elif isinstance(value, (np.generic, np.ndarray)): # Handle NumPy types
                        # Convert NumPy types to native Python types (scalar or list)
                        cleaned_hole_data[key] = value.item() if value.size == 1 else value.tolist()
                    elif isinstance(value, pd.Series): # Handle potential Series objects
                         # Assuming the Series only contains one value or we only need the first
                        cleaned_hole_data[key] = value.iloc[0] if not value.empty and not pd.isna(value.iloc[0]) else None
                    else:
                        cleaned_hole_data[key] = value # Keep other types as is

                # Reference to the hole document in the 'holes' subcollection
                hole_doc_ref = holes_subcollection_ref.document(hole_num_str)

                # Set the data in Firestore
                try:
                    hole_doc_ref.set(cleaned_hole_data)
                except Exception as e:
                    print(f"    ❌ Error uploading hole {hole_num_str} for user {player}, course {cc}: {e}")

        except Exception as e:
            print(f"  ❌ Error accessing/processing user {player}, course {cc}: {e}")

print(f"\n✅ Upload of data to 'users/{{userId}}/course_summaries/{{courseId}}' and 'users/{{userId}}/course_summaries/{{courseId}}/holes/{{holeNumber}}' complete!")

Iterating through 44 unique user-course combinations to upload data...
  ❌ Error accessing/processing user ML, course ARRAYANES: 403 Permission denied on resource project golf-scorecard-1847c. [reason: "CONSUMER_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "firestore.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "golf-scorecard-1847c"
}
metadata {
  key: "consumer"
  value: "projects/golf-scorecard-1847c"
}
, locale: "en-US"
message: "Permission denied on resource project golf-scorecard-1847c."
, links {
  description: "Google developers console"
  url: "https://console.developers.google.com"
}
]
  ❌ Error accessing/processing user ML, course BRICEÑO: 403 Permission denied on resource project golf-scorecard-1847c. [reason: "CONSUMER_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "firestore.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "golf-scorecard-1847c"
}
metadata {
  key: "consumer"
  value: "projects/

## Review and refine

### Subtask:
Review the data structure in Firestore after the attempted upload to understand the current state and identify discrepancies based on the desired structure, given the permission errors encountered.


## Review and refine retry 1

### Subtask:
Review the data structure in Firestore after the attempted upload to understand the current state and identify discrepancies based on the desired structure. Given the previous permission errors, focus on verifying if any documents were created at all, or if the error prevented any writes. This step requires manual review via the Firebase console.


## Summary:

### Data Analysis Key Findings

*   The process successfully identified 'PLAYER' as the `userId` and 'CC' as the `courseId` from the input dataframes (`deposit_df` and `index_df`) for structuring the Firestore data.
*   The code was correctly modified to group the `merged_hole_data` by user and course, aligning with the intended data organization for Firestore.
*   The code was updated to construct the correct Firestore paths: `/users/{userId}/course_summaries/{courseId}` for course summaries and `/users/{userId}/course_summaries/{courseId}/holes/{holeNumber}` for hole-by-hole data.
*   The code implemented the logic to aggregate course summary data (number of rounds and average score) and attempted to upload it to the corresponding Firestore document path.
*   The code implemented the logic to iterate through grouped hole data, format it, and attempted to upload each hole's data to the 'holes' subcollection under the course summary document.
*   During the attempted Firestore uploads, repeated "403 Permission denied" errors were encountered, indicating an issue with the service account's permissions to write to the Firestore project.

### Insights or Next Steps

*   The primary blocker for successful data upload is the "403 Permission denied" error. The next crucial step is to verify and update the service account permissions in the Google Cloud Platform console to ensure it has the necessary roles (e.g., Cloud Datastore User or Owner) to write data to Firestore.
*   After resolving the permission issues, re-execute the data upload script to confirm that data is successfully written to Firestore with the desired structure. A manual review of the Firestore database via the Firebase console will be necessary to confirm the successful upload and correct data structure.


# Task
Provide a step-by-step guide on how to integrate Google Drive, GitHub, Colab, and Firebase for data uploading, backup, and live server functions, focusing on a "practice mode" setup.

## Set up google cloud project and firebase

### Subtask:
Ensure you have a Google Cloud Project and a Firebase project set up and linked. This is where your Firestore database will reside.


In [1]:
# Install the Firebase Admin SDK
!pip install firebase-admin

In [4]:
import firebase_admin
from firebase_admin import credentials, firestore
import os

# Replace with the actual path to your service account key JSON file in Google Drive
# Make sure your Google Drive is mounted in Colab.
service_account_key_path = "/content/drive/MyDrive/serviceAccountKey.json"

# Initialize Firebase Admin SDK if it hasn't been initialized yet
try:
    # Check if a default app is already initialized to avoid re-initialization errors
    firebase_admin.get_app()
    print("Firebase Admin SDK already initialized.")
except ValueError:
    # If not initialized, create credentials and initialize the app
    if not os.path.exists(service_account_key_path):
        print(f"❌ Error: Service account key not found at '{service_account_key_path}'. Please check the path and ensure your Google Drive is mounted.")
    else:
        try:
            cred = credentials.Certificate(service_account_key_path)
            firebase_admin.initialize_app(cred)
            print("✅ Firebase Admin SDK initialized successfully.")
        except Exception as e:
            print(f"❌ Error initializing Firebase Admin SDK: {e}")

# Get the Firestore client
try:
    db = firestore.client()
    print("✅ Firestore client obtained successfully.")
except Exception as e:
    print(f"❌ Error getting Firestore client: {e}")

✅ Firebase Admin SDK initialized successfully.
✅ Firestore client obtained successfully.


## Configure Firebase Admin SDK

### Subtask:
Set up and configure the Firebase Admin SDK in your Colab environment using the generated service account key.